# Set up environment

**Import libraries**

In [127]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.data import Dataset
from tensorflow.keras import callbacks
import numpy as np
import cv2
import copy
import time

In [110]:
def image_pooling(image, new_width, new_height):
    return cv2.resize(image, (new_height, new_width), interpolation=cv2.INTER_AREA)

def computeRollingVarianceSum(mean, prev_mean, cur, last):
    return  (cur - prev_mean) * (cur - mean) - (last - prev_mean) * (last - mean)

def swapPositions(li, pos1, pos2):
    li[pos1],li[pos2] = li[pos2],li[pos1]
    return li
def push_down(li):
    for i in range(len(li)-1):
        swapPositions(li, i, i+1)
     #list[-1] is untouched
    return li

In [145]:
def getVariances(video_path, frame_width=128, frame_height=128, target_frame_width=28, target_frame_height=28):
    """
    video_path is the file path to the input video
    Uses functions in the github

    Returns:
    (diff_variances, input_variances)
    """

    rotation_frequency = 150  # enter in revolution per second

    # say the input frames are periodic and can be described by a*sin(bx+c)+d
    # its derivative is a*b*cos(bx+c)
    # its second derivative is -a*b*b*sin(bx+c)
    # so f(x)-f'(x-pi/(2*b))/b should be constant so should f(x)+f"(x)/b/b
    # the second is much better because the first involves a phase shift which
    # requires access to data taken pi/(2*b) ago and will thus lower the processing
    # speed or accuracy.
    # the key is to know b, which is equal to frequency*2pi
    frequency_const = rotation_frequency*2*np.pi

    num_frame = 5

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(5)
    total_frames = int(cap.get(7))-num_frame+1
    print('Video loaded with', total_frames,'frames')

    input_frame = [np.zeros((target_frame_width,target_frame_height),dtype=np.float32) for i in range(num_frame+1)]
    input_mean = np.zeros((target_frame_width,target_frame_height),dtype=np.float32)
    input_var_sum = np.zeros((target_frame_width,target_frame_height),dtype=np.float32)

    derivative1 = [np.zeros((target_frame_width,target_frame_height),dtype=np.float32) for i in range(num_frame)]
    derivative2 = [np.zeros((target_frame_width,target_frame_height),dtype=np.float32) for i in range(num_frame-1)]

    diff = [np.zeros((target_frame_width,target_frame_height),dtype=np.float32) for i in range(num_frame-1)]
    diff_mean = np.zeros((target_frame_width,target_frame_height),dtype=np.float32)
    diff_var_sum = np.zeros((target_frame_width,target_frame_height),dtype=np.float32)

    input_vars = np.zeros((int(cap.get(7)),target_frame_width,target_frame_height),dtype=np.float32)
    diff_vars = np.zeros((int(cap.get(7)),target_frame_width,target_frame_height),dtype=np.float32)

    deltaTime = 1.0/fps
    for i in range(int(cap.get(7))):
        ret, frame = cap.read()
        assert ret
        # shallow copy !!!
        prev_input_mean = copy.deepcopy(input_mean)
        prev_diff_mean = copy.deepcopy(diff_mean)

        input_frame = push_down(input_frame)
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        frame = image_pooling(frame, target_frame_width, target_frame_height)
        del_input_frame = copy.deepcopy(input_frame[-1])
        input_frame[-1]=frame
        input_mean = (input_mean*(num_frame+1) + frame - del_input_frame)/(num_frame+1)
        # 1 2 3 3 4 6
        #  1 2 3 4 5
        #   3 4 5 6
        derivative1 = push_down(derivative1)
        derivative1[-1]=(input_frame[-1]-input_frame[-2])/deltaTime
        derivative2 = push_down(derivative2)
        derivative2[-1]=(derivative1[-1]-derivative1[-2])/deltaTime

        diff = push_down(diff)
        del_diff = copy.deepcopy(diff[-1])
        diff[-1] = input_frame[-2]+derivative2[-1]/(frequency_const**2)
        diff_mean = (diff_mean*(num_frame-1) + diff[-1] - del_diff)/(num_frame-1)


        input_var_sum += computeRollingVarianceSum(input_mean, prev_input_mean, input_frame[-1], del_input_frame)

        diff_var_sum += computeRollingVarianceSum(diff_mean, prev_diff_mean, diff[-1], del_diff)

        input_var_sum2 = np.var(np.asarray(input_frame), axis=0)
        diff_var_sum2 = np.var(np.asarray(diff), axis=0)

        input_vars[i] = input_var_sum/(num_frame+1)
        diff_vars[i] = diff_var_sum/(num_frame-1)
    return diff_vars, input_vars



In [32]:
def variancesToDataset(diff_variance,input_variance,window_location, batch_size=16):
    """
    diff_variance and input_variance are three dimensional numpy arrays, with third dimension being frame number
    Before inputting into this function, stack every frame from all images to diff_variance and input_variance
    window_location is binary for whether or not there is a window at a pixel in a certain frame

    Returns:
    Dataset
    """
    features = np.transpose(np.array((diff_variance.flatten(),input_variance.flatten())))
    data = (features,window_location.flatten())
    ds = Dataset.from_tensor_slices(data)
    return ds.batch(batch_size).prefetch(1)

In [33]:
def get2D(diff_variances, input_variances):
    """
    diff_variance and input_variance are three dimensional numpy arrays, with third dimension being frame number
    Before inputting into this function, stack every frame from all images to diff_variance and input_variance
    
    Returns:
    2d numpy array with [variance, variance] as each row
    """
    return np.transpose(np.array((diff_variances.flatten(),input_variances.flatten())))

Preprocessing

In [146]:
diff1, input1 = getVariances('outdoor_window_Trim.mp4')
diff2, input2 = getVariances('indoor.avi')

48521
0.0012675112811848521
0.0012675112811848521
0.0012675112811848521
0.002859387779608369
0.002859387779608369
0.005141638361111917
0.0009794009910170797
0.0007967128185555339
0.0009504577028565109
0.0011370218126103282
0.0024443911742904876
0.0011750811427666374
0.0010001990093838249
0.001009017630966461
0.0008575884276069701
0.0016491314163431525
0.008695036172866821
0.008695036172866821
0.008674247688423028
0.00266961008310318
0.002669032599331892
0.0020569826010614634
0.0020564051172901753
0.0020359767894466284
0.001191023620776832
0.001191023620776832
0.001191023620776832
0.0011414553737267852
0.0013806818751618266
0.0013806818751618266
0.0013801043913905385
0.0012970528656571405
0.0008434801711700857
-0.0006584972013274339
-0.7121370658578581
-1.04871950107108
-1.9140921617798383
-1.0395989359179505
-2.238608705728174
-4.628243015957743
-0.6949439784326821
-0.6879971613398246
-0.7135953671478463
-0.696171287650288
-0.0009314470863695379
-0.03487658280575591
0.00057490530889481

In [39]:
features1 = get2D(diff1,input1)
features2 = get2D(diff2,input2)
features2 = features2[:features1.shape[0]]
features = np.concatenate((features1, features2))
features[10]

array([       nan, 0.00023502])

In [28]:
labels1 = np.ones(features1.shape[0])
labels2 = np.zeros(features2.shape[0])
labels = np.concatenate((labels1,labels2))

In [34]:
model = Sequential([
    tf.keras.layers.Dense(2, activation='relu', input_shape=(2,)),
    layers.Dense(1, activation='sigmoid',input_shape=(2,))
])

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=["accuracy"])

NameError: name 'tf' is not defined

In [32]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [33]:
stopper = callbacks.EarlyStopping(monitor='val_accuracy',patience=5)
model.fit(
    features,
    labels,
    batch_size=64, 
    epochs=100, 
    validation_split=0.2,
    callbacks=[stopper]
)

Epoch 1/100
7507/7507 [==============================] - 7s 855us/step - loss: nan - accuracy: 0.3738 - val_loss: nan - val_accuracy: 1.0000
Epoch 2/100
7507/7507 [==============================] - 7s 869us/step - loss: nan - accuracy: 0.3748 - val_loss: nan - val_accuracy: 1.0000
Epoch 3/100
7503/7507 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.3741

KeyboardInterrupt: 

In [21]:
model.save('simpleLogistic')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: simpleLogistic/assets


In [22]:
output = model.predict(features[:10])
output.flatten()

array([0.00146401, 0.00052735, 0.00065494, 0.00053045, 0.00186622,
       0.00169724, 0.00168318, 0.00253487, 0.00287482, 0.00096923],
      dtype=float32)